<a href="https://colab.research.google.com/github/dk-wei/NLP-progress/blob/master/Text_Data_Cleaning_Medium.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [61]:
import re
import sklearn
import numpy as np
import sklearn
import sklearn.ensemble
import sklearn.metrics
from __future__ import print_function
from tqdm.notebook import tqdm

import spacy
import string
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [62]:
from sklearn.datasets import fetch_20newsgroups
newsgroups_train = fetch_20newsgroups(subset='train')
newsgroups_test = fetch_20newsgroups(subset='test')

In [63]:
newsgroups_train.data[20]

'From: keith@cco.caltech.edu (Keith Allan Schneider)\nSubject: Re: <<Pompous ass\nOrganization: California Institute of Technology, Pasadena\nLines: 16\nNNTP-Posting-Host: punisher.caltech.edu\n\nlivesey@solntze.wpd.sgi.com (Jon Livesey) writes:\n\n[...]\n>>The "`little\' things" above were in reference to Germany, clearly.  People\n>>said that there were similar things in Germany, but no one could name any.\n>That\'s not true.  I gave you two examples.  One was the rather\n>pevasive anti-semitism in German Christianity well before Hitler\n>arrived.  The other was the system of social ranks that were used\n>in Imperail Germany and Austria to distinguish Jews from the rest \n>of the population.\n\nThese don\'t seem like "little things" to me.  At least, they are orders\nworse than the motto.  Do you think that the motto is a "little thing"\nthat will lead to worse things?\n\nkeith\n'

In [64]:
compared_text = newsgroups_train.data[20]

In [65]:
print(newsgroups_train.data[20])

From: keith@cco.caltech.edu (Keith Allan Schneider)
Subject: Re: <<Pompous ass
Organization: California Institute of Technology, Pasadena
Lines: 16
NNTP-Posting-Host: punisher.caltech.edu

livesey@solntze.wpd.sgi.com (Jon Livesey) writes:

[...]
>>The "`little' things" above were in reference to Germany, clearly.  People
>>said that there were similar things in Germany, but no one could name any.
>That's not true.  I gave you two examples.  One was the rather
>pevasive anti-semitism in German Christianity well before Hitler
>arrived.  The other was the system of social ranks that were used
>in Imperail Germany and Austria to distinguish Jews from the rest 
>of the population.

These don't seem like "little things" to me.  At least, they are orders
worse than the motto.  Do you think that the motto is a "little thing"
that will lead to worse things?

keith



In [66]:
base_stop_words = stopwords.words("english")
punctuation_string = list(string.punctuation)
extra_stopwords_black = {'n/a', 'unspecified',"subject"}    # extra stopwords
extra_stopwords_white = {'build'}           # not stopwords
base_stop_words.extend(punctuation_string)
stopwords_eng = set(base_stop_words)
stopwords_eng.update(extra_stopwords_black)
stopwords_eng.difference_update(extra_stopwords_white)


In [67]:
stopwords_eng

{'!',
 '"',
 '#',
 '$',
 '%',
 '&',
 "'",
 '(',
 ')',
 '*',
 '+',
 ',',
 '-',
 '.',
 '/',
 ':',
 ';',
 '<',
 '=',
 '>',
 '?',
 '@',
 '[',
 '\\',
 ']',
 '^',
 '_',
 '`',
 'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'n/a'

In [68]:
import spacy
from spacy.tokenizer import Tokenizer
from spacy.util import compile_infix_regex

nlp = spacy.load("en_core_web_sm")

def custom_tokenizer(nlp):
    inf = list(nlp.Defaults.infixes)               # Default infixes
    inf.remove(r"(?<=[0-9])[+\-\*^](?=[0-9-])")    # Remove the generic op between numbers or between a number and a -
    inf = tuple(inf)                               # Convert inf to tuple
    infixes = inf + tuple([r"(?<=[0-9])[+*^](?=[0-9-])", r"(?<=[0-9])-(?=-)"])  # Add the removed rule after subtracting (?<=[0-9])-(?=[0-9]) pattern
    infixes = [x for x in infixes if '-|–|—|--|---|——|~' not in x] # Remove - between letters rule
    infix_re = compile_infix_regex(infixes)

    return Tokenizer(nlp.vocab, prefix_search=nlp.tokenizer.prefix_search,
                                suffix_search=nlp.tokenizer.suffix_search,
                                infix_finditer=infix_re.finditer,
                                token_match=nlp.tokenizer.token_match,
                                rules=nlp.Defaults.tokenizer_exceptions)

nlp.tokenizer = custom_tokenizer(nlp)


In [69]:
doc = nlp('The Indo-European Caucus won the all-male election 58-32.')

print([token.text for token in doc]) 

['The', 'Indo-European', 'Caucus', 'won', 'the', 'all-male', 'election', '58-32', '.']


In [78]:
def text_process(text_string):

  text_string = text_string.replace("/", ' ').replace(',', ' ').replace('?', ' ').replace('!', ' ').replace(':', ' ').strip()

  processed_token = [i.lower() for i in re.split('\s|\n|\t', text_string.strip()) if i != '']

  processed_token = list(filter (lambda s:any([c.isalnum() for c in s]), processed_token))

  processed_token = [i.rstrip('.').
                     rstrip(',').
                     lstrip('(').
                     rstrip(')').
                     lstrip('<').
                     rstrip('>').
                     rstrip(':').
                     lstrip('>').
                     rstrip('>').
                     lstrip('"').
                     rstrip('"').
                     lstrip("'").
                     rstrip("'").
                     lstrip("-").
                     rstrip("-") for i in processed_token]

  processed_token = ' '.join([i for i in processed_token if i not in stopwords_eng])

  # doc = nlp(processed_token)

  # return ' '.join([token.lemma_ for token in doc if token.text not in stopwords_eng])

  return processed_token

In [79]:
text_process(newsgroups_train.data[20])

"keith@cco.caltech.edu keith allan schneider pompous ass organization california institute technology pasadena lines 16 nntp-posting-host punisher.caltech.edu livesey@solntze.wpd.sgi.com jon livesey writes `little things reference germany clearly people said similar things germany one could name that's true gave two examples one rather pevasive anti-semitism german christianity well hitler arrived system social ranks used imperail germany austria distinguish jews rest population seem like little things least orders worse motto think motto little thing lead worse things keith"

In [53]:
compared_text

'From: keith@cco.caltech.edu (Keith Allan Schneider)\nSubject: Re: <<Pompous ass\nOrganization: California Institute of Technology, Pasadena\nLines: 16\nNNTP-Posting-Host: punisher.caltech.edu\n\nlivesey@solntze.wpd.sgi.com (Jon Livesey) writes:\n\n[...]\n>>The "`little\' things" above were in reference to Germany, clearly.  People\n>>said that there were similar things in Germany, but no one could name any.\n>That\'s not true.  I gave you two examples.  One was the rather\n>pevasive anti-semitism in German Christianity well before Hitler\n>arrived.  The other was the system of social ranks that were used\n>in Imperail Germany and Austria to distinguish Jews from the rest \n>of the population.\n\nThese don\'t seem like "little things" to me.  At least, they are orders\nworse than the motto.  Do you think that the motto is a "little thing"\nthat will lead to worse things?\n\nkeith\n'

In [72]:
text_string = compared_text.replace("/", ' ').replace(',', ' ').replace('?', ' ').replace('!', ' ').replace(':', ' ').strip()

text_string

'From  keith@cco.caltech.edu (Keith Allan Schneider)\nSubject  Re  <<Pompous ass\nOrganization  California Institute of Technology  Pasadena\nLines  16\nNNTP-Posting-Host  punisher.caltech.edu\n\nlivesey@solntze.wpd.sgi.com (Jon Livesey) writes \n\n[...]\n>>The "`little\' things" above were in reference to Germany  clearly.  People\n>>said that there were similar things in Germany  but no one could name any.\n>That\'s not true.  I gave you two examples.  One was the rather\n>pevasive anti-semitism in German Christianity well before Hitler\n>arrived.  The other was the system of social ranks that were used\n>in Imperail Germany and Austria to distinguish Jews from the rest \n>of the population.\n\nThese don\'t seem like "little things" to me.  At least  they are orders\nworse than the motto.  Do you think that the motto is a "little thing"\nthat will lead to worse things \n\nkeith'

In [73]:
#processed_token = ' '.join(re.split('\s|; |: |, |- |\* |\n, |\t', text_string.strip())).split()
processed_token = [i.lower() for i in re.split('\s|\n|\t', text_string.strip()) if i != '']
processed_token

['from',
 'keith@cco.caltech.edu',
 '(keith',
 'allan',
 'schneider)',
 'subject',
 're',
 '<<pompous',
 'ass',
 'organization',
 'california',
 'institute',
 'of',
 'technology',
 'pasadena',
 'lines',
 '16',
 'nntp-posting-host',
 'punisher.caltech.edu',
 'livesey@solntze.wpd.sgi.com',
 '(jon',
 'livesey)',
 'writes',
 '[...]',
 '>>the',
 '"`little\'',
 'things"',
 'above',
 'were',
 'in',
 'reference',
 'to',
 'germany',
 'clearly.',
 'people',
 '>>said',
 'that',
 'there',
 'were',
 'similar',
 'things',
 'in',
 'germany',
 'but',
 'no',
 'one',
 'could',
 'name',
 'any.',
 ">that's",
 'not',
 'true.',
 'i',
 'gave',
 'you',
 'two',
 'examples.',
 'one',
 'was',
 'the',
 'rather',
 '>pevasive',
 'anti-semitism',
 'in',
 'german',
 'christianity',
 'well',
 'before',
 'hitler',
 '>arrived.',
 'the',
 'other',
 'was',
 'the',
 'system',
 'of',
 'social',
 'ranks',
 'that',
 'were',
 'used',
 '>in',
 'imperail',
 'germany',
 'and',
 'austria',
 'to',
 'distinguish',
 'jews',
 'from',


In [74]:
processed_token = list(filter (lambda s:any([c.isalnum() for c in s]), processed_token))
processed_token

['from',
 'keith@cco.caltech.edu',
 '(keith',
 'allan',
 'schneider)',
 'subject',
 're',
 '<<pompous',
 'ass',
 'organization',
 'california',
 'institute',
 'of',
 'technology',
 'pasadena',
 'lines',
 '16',
 'nntp-posting-host',
 'punisher.caltech.edu',
 'livesey@solntze.wpd.sgi.com',
 '(jon',
 'livesey)',
 'writes',
 '>>the',
 '"`little\'',
 'things"',
 'above',
 'were',
 'in',
 'reference',
 'to',
 'germany',
 'clearly.',
 'people',
 '>>said',
 'that',
 'there',
 'were',
 'similar',
 'things',
 'in',
 'germany',
 'but',
 'no',
 'one',
 'could',
 'name',
 'any.',
 ">that's",
 'not',
 'true.',
 'i',
 'gave',
 'you',
 'two',
 'examples.',
 'one',
 'was',
 'the',
 'rather',
 '>pevasive',
 'anti-semitism',
 'in',
 'german',
 'christianity',
 'well',
 'before',
 'hitler',
 '>arrived.',
 'the',
 'other',
 'was',
 'the',
 'system',
 'of',
 'social',
 'ranks',
 'that',
 'were',
 'used',
 '>in',
 'imperail',
 'germany',
 'and',
 'austria',
 'to',
 'distinguish',
 'jews',
 'from',
 'the',
 '

In [75]:
processed_token = [i.rstrip('.').
                     rstrip(',').
                     lstrip('(').
                     rstrip(')').
                     lstrip('<').
                     rstrip('>').
                     rstrip(':').
                     lstrip('>').
                     rstrip('>').
                     lstrip('"').
                     rstrip('"').
                     lstrip("'").
                     rstrip("'").
                     lstrip("-").
                     rstrip("-") for i in processed_token]

processed_token

['from',
 'keith@cco.caltech.edu',
 'keith',
 'allan',
 'schneider',
 'subject',
 're',
 'pompous',
 'ass',
 'organization',
 'california',
 'institute',
 'of',
 'technology',
 'pasadena',
 'lines',
 '16',
 'nntp-posting-host',
 'punisher.caltech.edu',
 'livesey@solntze.wpd.sgi.com',
 'jon',
 'livesey',
 'writes',
 'the',
 '`little',
 'things',
 'above',
 'were',
 'in',
 'reference',
 'to',
 'germany',
 'clearly',
 'people',
 'said',
 'that',
 'there',
 'were',
 'similar',
 'things',
 'in',
 'germany',
 'but',
 'no',
 'one',
 'could',
 'name',
 'any',
 "that's",
 'not',
 'true',
 'i',
 'gave',
 'you',
 'two',
 'examples',
 'one',
 'was',
 'the',
 'rather',
 'pevasive',
 'anti-semitism',
 'in',
 'german',
 'christianity',
 'well',
 'before',
 'hitler',
 'arrived',
 'the',
 'other',
 'was',
 'the',
 'system',
 'of',
 'social',
 'ranks',
 'that',
 'were',
 'used',
 'in',
 'imperail',
 'germany',
 'and',
 'austria',
 'to',
 'distinguish',
 'jews',
 'from',
 'the',
 'rest',
 'of',
 'the',
 

In [76]:
processed_token = ' '.join([i for i in processed_token if i not in stopwords_eng])

processed_token

"keith@cco.caltech.edu keith allan schneider pompous ass organization california institute technology pasadena lines 16 nntp-posting-host punisher.caltech.edu livesey@solntze.wpd.sgi.com jon livesey writes `little things reference germany clearly people said similar things germany one could name that's true gave two examples one rather pevasive anti-semitism german christianity well hitler arrived system social ranks used imperail germany austria distinguish jews rest population seem like little things least orders worse motto think motto little thing lead worse things keith"

In [77]:
nlp = spacy.load("en_core_web_sm")

doc = nlp(processed_token)

' '.join([token.lemma_ for token in doc if token.text not in stopwords_eng])

'keith@cco.caltech.edu keith allan schneider pompous ass organization california institute technology pasadena line 16 nntp post host punisher.caltech.edu livesey@solntze.wpd.sgi.com jon livesey write little thing reference germany clearly people say similar thing germany one could name be true give two example one rather pevasive anti semitism german christianity well hitler arrive system social rank use imperail germany austria distinguish jews rest population seem like little thing least order bad motto think motto little thing lead bad thing keith'

In [33]:
compared_text

'From: keith@cco.caltech.edu (Keith Allan Schneider)\nSubject: Re: <<Pompous ass\nOrganization: California Institute of Technology, Pasadena\nLines: 16\nNNTP-Posting-Host: punisher.caltech.edu\n\nlivesey@solntze.wpd.sgi.com (Jon Livesey) writes:\n\n[...]\n>>The "`little\' things" above were in reference to Germany, clearly.  People\n>>said that there were similar things in Germany, but no one could name any.\n>That\'s not true.  I gave you two examples.  One was the rather\n>pevasive anti-semitism in German Christianity well before Hitler\n>arrived.  The other was the system of social ranks that were used\n>in Imperail Germany and Austria to distinguish Jews from the rest \n>of the population.\n\nThese don\'t seem like "little things" to me.  At least, they are orders\nworse than the motto.  Do you think that the motto is a "little thing"\nthat will lead to worse things?\n\nkeith\n'

In [52]:
s = "I enjoyd the event which took place yesteday &amp; I luvd it ! The link to the show is http://t.co/4ftYom0i It's awsome you'll luv it #HadFun #Enjoyed BFN GN"

text_process(s)

'enjoyd event took place yesteday &amp luvd link show http t.co 4ftyom0i awsome luv #hadfun #enjoyed bfn gn'